# 0. 라이브러리 설치

In [1]:
!pip install pycaret==2.3.10
!pip install jinja2==3.1.2
!pip install xgboost==1.6.0

     |████████████████████████████████| 125 kB 14.3 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.2
    Uninstalling Jinja2-3.1.2:
      Successfully uninstalled Jinja2-3.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.2

# 1. 데이터 불러오기

In [2]:
import io
import pandas as pd

In [3]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/train.csv')
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [6]:
train_y = train.credit
train_y

0        1.0
1        1.0
2        2.0
3        0.0
4        2.0
        ... 
26452    1.0
26453    2.0
26454    2.0
26455    2.0
26456    2.0
Name: credit, Length: 26457, dtype: float64

# 2. 데이터 전처리

In [7]:
df = train.drop(['index','FLAG_MOBIL','credit'], axis = 1)
df.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,NaN,2.0,-6.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0


In [8]:
train_notnull_df = df[df.occyp_type.notnull()]
train_null_df = df[df.occyp_type.isna()]

# Pycaret

In [9]:
!pip install catboost

In [10]:
from pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [11]:
model = setup(data= train_notnull_df,
              target='occyp_type',
              numeric_features=['DAYS_EMPLOYED','child_num'], # categoric으로 분류돼서 직접 지정
              train_size=0.7,
              session_id=42)

,Description,Value
0,session_id,42
1,Target,occyp_type
2,Target Type,Multiclass
3,Label Encoded,"Accountants: 0, Cleaning staff: 1, Cooking sta..."
4,Original Data,"(18286, 17)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [12]:
# 평가 지표 추가
from sklearn.metrics import log_loss
add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f94766c7440>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [13]:
# svm과 ridge 모델은 predict_proba를 지원하지 않아 제외
top_3_models = compare_models(sort='Accuracy', n_select=3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
et,Extra Trees Classifier,0.8551,0.9742,0.8173,0.8575,0.8547,0.8324,0.8327,1.0132,1.691
rf,Random Forest Classifier,0.8515,0.9707,0.8037,0.8559,0.8512,0.8279,0.8284,0.9703,1.555
dt,Decision Tree Classifier,0.8109,0.8908,0.7781,0.8129,0.8106,0.7821,0.7823,6.5327,0.101
xgboost,Extreme Gradient Boosting,0.7960,0.9557,0.7511,0.8082,0.7962,0.7626,0.7638,0.8080,23.520
lightgbm,Light Gradient Boosting Machine,0.7534,0.9256,0.7128,0.7691,0.7552,0.7143,0.7157,1.8364,2.971
catboost,CatBoost Classifier,0.7355,0.9403,0.6904,0.7523,0.7350,0.6911,0.6930,0.9725,43.860
knn,K Neighbors Classifier,0.5911,0.8842,0.5436,0.5958,0.5901,0.5286,0.5290,4.9492,0.187
gbc,Gradient Boosting Classifier,0.5355,0.8614,0.5525,0.5775,0.5298,0.4513,0.4558,1.4677,25.338
lda,Linear Discriminant Analysis,0.3477,0.7536,0.1659,0.3113,0.3154,0.2251,0.2293,1.9881,0.095
nb,Naive Bayes,0.2669,0.6246,0.0812,0.1760,0.1731,0.0726,0.0936,2.3142,0.051


In [14]:
predict_model(top_3_models[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Extra Trees Classifier,0.8669,0.9768,0.8226,0.8693,0.8668,0.8453,0.8455,0.9279


,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,gender_F,car_N,reality_N,income_type_Commercial associate,...,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,work_phone_0,phone_0,email_0,occyp_type,Label,Score
0,0.0,144000.0,-18136.0,-10110.0,2.0,-22.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,High skill tech staff,High skill tech staff,0.98
1,0.0,562500.0,-14654.0,-2836.0,2.0,-11.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,Managers,Managers,1.00
2,0.0,240750.0,-17693.0,-7778.0,2.0,-27.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,Laborers,Laborers,1.00
3,0.0,270000.0,-16668.0,-377.0,1.0,-7.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,Sales staff,Sales staff,0.98
4,0.0,270000.0,-13554.0,-3965.0,2.0,-57.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Managers,Managers,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5481,0.0,157500.0,-19843.0,-1922.0,2.0,-27.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,Managers,Managers,0.70
5482,1.0,103500.0,-11529.0,-651.0,2.0,-1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,Accountants,Accountants,0.75
5483,0.0,211500.0,-10072.0,-1101.0,1.0,-10.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Core staff,Core staff,0.94
5484,0.0,180000.0,-10138.0,-224.0,1.0,-42.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,Laborers,Security staff,0.28


### 파라미터 튜닝

In [15]:
tuned_top3 = [tune_model(i, optimize='Accuracy') for i in top_3_models]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.5547,0.8815,0.3749,0.5638,0.5461,0.4795,0.4828,3.0453
1,0.5422,0.8775,0.3820,0.5500,0.5336,0.4665,0.4691,3.2682
2,0.5492,0.8697,0.3435,0.5657,0.5383,0.4724,0.4771,3.8792
3,0.5195,0.8686,0.3563,0.5208,0.5107,0.4389,0.4399,3.6784
4,0.5539,0.8784,0.3632,0.5656,0.5466,0.4796,0.4828,3.3455
5,0.5156,0.8746,0.3739,0.5239,0.5106,0.4399,0.4427,3.2046
6,0.5242,0.8684,0.3280,0.5268,0.5148,0.4449,0.4466,3.9090
7,0.5453,0.8659,0.3672,0.5597,0.5393,0.4709,0.4741,3.6847
8,0.5547,0.8767,0.4429,0.5606,0.5498,0.4803,0.4817,3.7256


In [16]:
tuned_top3

[ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                      criterion='entropy', max_depth=9, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=230, n_jobs=-1,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                        criterion='entropy', max_depth=9, max_features='sqrt',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0, min_impurity_split=None,
                        min_samples_leaf=6, min_samples_split=10,
                        min_weight_fraction_leaf=0.0, n_estimators=230,
                        n_jobs=-1, oob_sc

### 모델 블랜딩

In [17]:
blend_top_3 = blend_models(estimator_list=tuned_top3, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.5695,0.8981,0.3577,0.5956,0.5525,0.4893,0.4956,1.4194
1,0.5758,0.8999,0.3450,0.5959,0.5598,0.4975,0.5034,1.4177
2,0.5805,0.8949,0.3409,0.6127,0.5629,0.5029,0.5091,1.4328
3,0.5562,0.8939,0.3333,0.5732,0.5365,0.4717,0.4776,1.4614
4,0.5734,0.9003,0.3483,0.6107,0.5581,0.4947,0.5003,1.4319
5,0.5664,0.8933,0.3747,0.6005,0.5511,0.4871,0.4933,1.4445
6,0.5758,0.8943,0.3387,0.6095,0.5567,0.4950,0.5011,1.4296
7,0.5625,0.8918,0.3336,0.6089,0.5461,0.4804,0.4873,1.4481
8,0.5852,0.8939,0.4102,0.6164,0.5719,0.5070,0.5137,1.4237


In [18]:
# 블렌딩 모델로 예측
pred = predict_model(blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.5815,0.9013,0.3551,0.6073,0.5699,0.5021,0.5074,1.3939


In [29]:
# Tuning 하지 않은 것 블렌딩
vanilla_blend_top_3 = blend_models(estimator_list=top_3_models, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.8430,0.9748,0.8337,0.8466,0.8432,0.8190,0.8191,0.7687
1,0.8391,0.9796,0.7954,0.8412,0.8391,0.8148,0.8149,0.6775
2,0.8289,0.9731,0.8016,0.8308,0.8290,0.8027,0.8028,0.7217
3,0.8289,0.9752,0.7961,0.8303,0.8287,0.8028,0.8030,0.7474
4,0.8430,0.9754,0.8145,0.8441,0.8423,0.8188,0.8190,0.8111
5,0.8500,0.9771,0.8295,0.8523,0.8498,0.8273,0.8276,0.6809
6,0.8266,0.9707,0.8006,0.8291,0.8266,0.7997,0.8000,0.8684
7,0.8469,0.9766,0.7602,0.8489,0.8462,0.8232,0.8233,0.8737
8,0.8133,0.9687,0.7550,0.8128,0.8123,0.7848,0.7849,0.9480


In [30]:
# Tuning하지 않은 블렌딩 모델로 예측 (성능 더 좋으므로 vanilla_blend_top_3로 예측)
pred = predict_model(vanilla_blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.8452,0.9769,0.8152,0.8455,0.8451,0.8207,0.8207,0.7178


### 최종 모델

In [31]:
# finalize_model : 전체 데이터로 마지막 학습을 진행함
final_blended = finalize_model(vanilla_blend_top_3)

In [32]:
save_model(final_blended, '/content/drive/MyDrive/Likelion_Semi_project_03/code/JW/final_van_blended_occyp1')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['DAYS_EMPLOYED',
                                                           'child_num'],
                                       target='occyp_type', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_val...
                                                                       class_weight=None,
                                                                       criterion='gini',
                                                                     

In [33]:
save_model(final_blended, 'final_van_blended_occyp1')
# files.download('final_blended_occyp1.pkl')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['DAYS_EMPLOYED',
                                                           'child_num'],
                                       target='occyp_type', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_val...
                                                                       class_weight=None,
                                                                       criterion='gini',
                                                                     

In [34]:
loaded_model = load_model('final_van_blended_occyp1')

Transformation Pipeline and Model Successfully Loaded


In [35]:
train_null_X = train_null_df.drop(['occyp_type'], axis=1)

In [36]:
# 최종 모델로 예측
prediction = predict_model(loaded_model, data=train_null_X)
prediction

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,Label,Score
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,2.0,-6.0,Accountants,0.4333
8,M,Y,Y,1,180000.0,Commercial associate,Higher education,Married,House / apartment,-15131,-1466,0,0,1,3.0,-38.0,Managers,0.4000
14,F,N,Y,0,157500.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-22922,365243,0,1,0,1.0,-41.0,Cleaning staff,0.4067
18,F,N,Y,0,216000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-23113,365243,0,0,0,2.0,-37.0,Cleaning staff,0.4767
19,F,N,Y,0,180000.0,Working,Secondary / secondary special,Married,House / apartment,-13727,-6031,0,0,0,2.0,-7.0,Cooking staff,0.3367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26439,F,N,Y,0,67500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-19238,365243,0,0,0,2.0,-57.0,Laborers,0.4533
26441,F,N,Y,0,90000.0,Pensioner,Incomplete higher,Separated,House / apartment,-24129,365243,0,0,0,1.0,-43.0,Laborers,0.4333
26443,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-22286,365243,0,0,0,1.0,-21.0,Cooking staff,0.4200
26449,F,N,N,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-10498,-2418,1,1,0,2.0,-2.0,Sales staff,0.6367


In [37]:
train_null_df.occyp_type = prediction.Label

In [38]:
train_null_df

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,Accountants,2.0,-6.0
8,M,Y,Y,1,180000.0,Commercial associate,Higher education,Married,House / apartment,-15131,-1466,0,0,1,Managers,3.0,-38.0
14,F,N,Y,0,157500.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-22922,365243,0,1,0,Cleaning staff,1.0,-41.0
18,F,N,Y,0,216000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-23113,365243,0,0,0,Cleaning staff,2.0,-37.0
19,F,N,Y,0,180000.0,Working,Secondary / secondary special,Married,House / apartment,-13727,-6031,0,0,0,Cooking staff,2.0,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26439,F,N,Y,0,67500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-19238,365243,0,0,0,Laborers,2.0,-57.0
26441,F,N,Y,0,90000.0,Pensioner,Incomplete higher,Separated,House / apartment,-24129,365243,0,0,0,Laborers,1.0,-43.0
26443,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-22286,365243,0,0,0,Cooking staff,1.0,-21.0
26449,F,N,N,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-10498,-2418,1,1,0,Sales staff,2.0,-2.0


In [46]:
final_train_df = pd.concat([train_notnull_df, train_null_df], axis=0)
final_train_df

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0
5,F,N,Y,2,270000.0,Working,Secondary / secondary special,Married,House / apartment,-13413,-4996,0,0,1,High skill tech staff,4.0,-18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26439,F,N,Y,0,67500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-19238,365243,0,0,0,Laborers,2.0,-57.0
26441,F,N,Y,0,90000.0,Pensioner,Incomplete higher,Separated,House / apartment,-24129,365243,0,0,0,Laborers,1.0,-43.0
26443,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Single / not married,House / apartment,-22286,365243,0,0,0,Cooking staff,1.0,-21.0
26449,F,N,N,0,90000.0,Working,Secondary / secondary special,Married,House / apartment,-10498,-2418,1,1,0,Sales staff,2.0,-2.0


In [47]:
final_train_df.sort_index(inplace=True)
final_train_df

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,Accountants,2.0,-6.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,0,0,0,Core staff,4.0,-2.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,0,0,0,Managers,2.0,-47.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,0,0,0,Core staff,2.0,-25.0
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,0,0,0,Laborers,1.0,-59.0


In [48]:
final_train_df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            26447, 26448, 26449, 26450, 26451, 26452, 26453, 26454, 26455,
            26456],
           dtype='int64', length=26457)

In [49]:
final_train_df_ = pd.concat([final_train_df, train_y], axis=1)
final_train_df_

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,Accountants,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,0,0,0,Core staff,4.0,-2.0,1.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,0,0,0,Managers,2.0,-47.0,2.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,0,0,0,Core staff,2.0,-25.0,2.0
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,0,0,0,Laborers,1.0,-59.0,2.0


In [50]:
final_train_df_.to_csv('/content/drive/MyDrive/Likelion_Semi_project_03/code/JW/train_occpy_pred_final.csv')

## Test data 예측

In [51]:
test_df = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/test.csv')
test_df.drop(['index','FLAG_MOBIL'], axis = 1, inplace = True)
test_df.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,0,1,0,NaN,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,0,Laborers,2.0,-40.0
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,0,0,Drivers,2.0,-41.0
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,0,0,Managers,2.0,-8.0


In [54]:
test_notnull_X

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,0,1,0,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,0,2.0,-40.0
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,0,0,2.0,-41.0
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,0,0,2.0,-8.0
5,F,Y,Y,0,126000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-18458,-2104,0,0,0,1.0,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,F,Y,Y,0,270000.0,Commercial associate,Higher education,Married,House / apartment,-19960,-5679,0,0,0,2.0,-31.0
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,-5434,1,1,0,2.0,-19.0
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,-1315,1,0,0,2.0,-34.0
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,-14018,0,0,0,2.0,-55.0


In [52]:
test_notnull_df = test_df[test_df.occyp_type.notnull()]
test_notnull_X = test_notnull_df.drop(['occyp_type'], axis=1)
test_notnull_y = test_notnull_df.occyp_type

test_null_df = test_df[test_df.occyp_type.isna()]
test_null_X = test_null_df.drop(['occyp_type'], axis=1)

In [57]:
test_notnull_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6848 entries, 1 to 9999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         6848 non-null   object 
 1   car            6848 non-null   object 
 2   reality        6848 non-null   object 
 3   child_num      6848 non-null   int64  
 4   income_total   6848 non-null   float64
 5   income_type    6848 non-null   object 
 6   edu_type       6848 non-null   object 
 7   family_type    6848 non-null   object 
 8   house_type     6848 non-null   object 
 9   DAYS_BIRTH     6848 non-null   int64  
 10  DAYS_EMPLOYED  6848 non-null   int64  
 11  work_phone     6848 non-null   int64  
 12  phone          6848 non-null   int64  
 13  email          6848 non-null   int64  
 14  occyp_type     6848 non-null   object 
 15  family_size    6848 non-null   float64
 16  begin_month    6848 non-null   float64
dtypes: float64(3), int64(6), object(8)
memory usage: 963

In [58]:
# 최종 모델로 예측
pred_test = predict_model(loaded_model, data=test_null_X)
pred_test

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,Label,Score
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,0,1,0,2.0,-60.0,Security staff,0.4033
6,F,N,Y,0,585000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-18858,-2010,0,1,0,2.0,-1.0,Laborers,0.4267
7,F,N,Y,0,141750.0,Pensioner,Higher education,Married,House / apartment,-20345,365243,0,1,0,2.0,-10.0,Sales staff,0.4600
8,F,N,N,0,225000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-19316,-4451,1,1,0,1.0,-23.0,Laborers,0.5733
10,F,N,Y,0,90000.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-21056,365243,0,1,0,1.0,-47.0,Cleaning staff,0.4367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,F,N,N,0,117000.0,Pensioner,Secondary / secondary special,Widow,House / apartment,-19698,365243,0,0,0,1.0,-27.0,Laborers,0.4600
9990,F,N,Y,0,157500.0,Working,Secondary / secondary special,Separated,House / apartment,-18148,-8376,0,0,0,1.0,-14.0,Medicine staff,0.3800
9992,F,N,Y,0,180000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-22655,365243,0,0,0,2.0,-30.0,Laborers,0.3867
9994,M,Y,Y,0,131400.0,Pensioner,Secondary / secondary special,Married,House / apartment,-23646,365243,0,0,0,2.0,-3.0,Security staff,0.6067


In [59]:
test_null_df.occyp_type = pred_test.Label

In [62]:
final_test_df = pd.concat([test_notnull_df, test_null_df], axis=0)
final_test_df.sort_index(inplace=True)
final_test_df

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,0,1,0,Security staff,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,0,Laborers,2.0,-40.0
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,0,0,Drivers,2.0,-41.0
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,0,0,Managers,2.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,-5434,1,1,0,Accountants,2.0,-19.0
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,-1315,1,0,0,Laborers,2.0,-34.0
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,-14018,0,0,0,Medicine staff,2.0,-55.0
9998,F,Y,N,0,180000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-16541,-1085,0,1,0,High skill tech staff,2.0,-33.0


In [63]:
final_test_df.to_csv('/content/drive/MyDrive/Likelion_Semi_project_03/code/JW/test_occpy_pred_final.csv')